## 1.0 Setting up Data Extraction

1.1 Initialization

In [1]:
import influxdb
from datetime import datetime
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Connecting to influxDB
HOST = 'influx.linklab.virginia.edu'
PORT = 443
USERNAME = 'sahbf24'
PASSWORD = 'raighee7Ahpheej3eud2sheob7seey7'
DATABASE = 'gateway-generic'

client = influxdb.InfluxDBClient(HOST, PORT, USERNAME, PASSWORD, DATABASE, ssl=True, verify_ssl=True)

1.2 Finding out how far back the data goes

In [18]:
query = """
SELECT FIRST("value") AS first_value
FROM "energy_kWh"
WHERE "location_specific" =~ /^([0-9]+ Olsson)$/
GROUP BY "location_specific"
"""
result = client.query(query)

for (measurement, tags), points in result.items():
    points = list(points)  # convert generator → list
    if not points:
        continue

    room = tags.get('location_specific', 'Unknown') if tags else 'Unknown'
    t = points[0]['time']
    v = points[0]['first_value']
    print(f"{room}: first at {t}, value={v}")

200 Olsson: first at 2020-09-18T20:01:00Z, value=2.12270188331604
201 Olsson: first at 2020-09-18T20:01:00Z, value=368.573974609375
203 Olsson: first at 2020-09-18T20:01:00Z, value=5943.81787109375
206 Olsson: first at 2020-09-18T20:01:00Z, value=3426.233642578125
208 Olsson: first at 2020-09-18T20:01:00Z, value=1454.7528076171875
210 Olsson: first at 2020-09-18T20:01:00Z, value=287.4942626953125
211 Olsson: first at 2020-09-18T20:01:00Z, value=950.355712890625
213 Olsson: first at 2020-09-18T20:01:00Z, value=3459.76123046875
217 Olsson: first at 2020-09-18T20:01:00Z, value=299.2487487792969
218 Olsson: first at 2020-09-18T20:01:00Z, value=170.62396240234375
219 Olsson: first at 2020-09-18T20:01:00Z, value=20.603864669799805
225 Olsson: first at 2020-09-18T20:01:00Z, value=356.97100830078125
229 Olsson: first at 2020-09-18T20:01:00Z, value=300.2880554199219
240 Olsson: first at 2020-09-18T20:01:00Z, value=1299.099853515625
241 Olsson: first at 2020-09-18T20:01:00Z, value=166.8680877685

It looks like the energy data starts at 8pm on September 18, 2020 for all energy sensors in the LLL. Since we are looking at historical trends and querying data by year, we will query all historical data from [2021-2024] inclusive, and also get all data for 2025 up through September and store them in .csv files. The data will be summed by `location_specific` across each week and stored as a row entry.

## 2.0 Getting all necessary data

2.1 Creating a function that accepts variable time limits

In [ ]:
def get_energy_summary(client, start_time, end_time):
    """
    Query energy data from InfluxDB and return a weekly summed DataFrame.

    Parameters
    ----------
    client : InfluxDBClient
        The connected InfluxDB client.
    start_time : str
        Start time in ISO format, e.g. '2024-01-01T00:00:00Z'.
    end_time : str
        End time in ISO format, e.g. '2025-01-01T00:00:00Z'.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns: ['time', 'location_specific', 'energy_sum', 'week']
    """
    query = f"""
    SELECT SUM("value") 
    FROM "energy_kWh"
    WHERE time >= '{start_time}' 
      AND time < '{end_time}'
      AND "location_specific" =~ /^([0-9]+ Olsson)$/
    GROUP BY time(1w), "location_specific" fill(0)
    """

    result = client.query(query)

    rows = []
    for (measurement, tags), points in result.items():
        location = tags.get('location_specific', 'Unknown')
        for p in points:
            rows.append({
                'time': pd.to_datetime(p['time']),
                'location_specific': location,
                'energy_MWh': round(p.get('sum', 0) / 1000, 3) # Convert kWh to MWh
            })

    df = pd.DataFrame(rows)
    df.sort_values(['location_specific', 'time'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Add an incremental week counter (starts at 1)
    df['week'] = df.groupby('location_specific').cumcount() + 1

    return df

test = get_energy_summary(client, '2024-01-01T00:00:00Z', '2025-01-01T00:00:00Z')
test.head()

,time,location_specific,energy_MWh,week
0,2023-12-28 00:00:00+00:00,200 Olsson,56.027,1
1,2024-01-04 00:00:00+00:00,200 Olsson,137.426,2
2,2024-01-11 00:00:00+00:00,200 Olsson,122.733,3
3,2024-01-18 00:00:00+00:00,200 Olsson,125.505,4
4,2024-01-25 00:00:00+00:00,200 Olsson,95.266,5


2.2 Finding where "reliable" data starts (aka not many missing entries)

In [31]:
energy_range = get_energy_summary(client, '2021-01-01T00:00:00Z', '2023-01-01T00:00:00Z')
energy_range

,time,location_specific,energy_MWh,week
0,2020-12-31 00:00:00+00:00,200 Olsson,11.370,1
1,2021-01-07 00:00:00+00:00,200 Olsson,11.123,2
2,2021-01-14 00:00:00+00:00,200 Olsson,0.000,3
3,2021-01-21 00:00:00+00:00,200 Olsson,0.000,4
4,2021-01-28 00:00:00+00:00,200 Olsson,0.000,5
...,...,...,...,...
4300,2022-12-01 00:00:00+00:00,286 Olsson,4097.280,101
4301,2022-12-08 00:00:00+00:00,286 Olsson,4187.623,102
4302,2022-12-15 00:00:00+00:00,286 Olsson,4095.209,103
4303,2022-12-22 00:00:00+00:00,286 Olsson,4135.238,104


Based on running the code chunk above, 2021 still has little consistent sensor data (covid? still setting up? unreliable power?). Therefore, our historical tracking will begin on Jan 1, 2022 and run till the end of September 2025. We will need to figure out how to deal with missing values, likely through some kind of average imputing.

2.3 Getting the data for our time range and storing it in a .csv file

In [32]:
energy_data = get_energy_summary(client, '2022-01-01T00:00:00Z', '2025-10-01T00:00:00Z')
energy_data.to_csv('energy_data_2022_2025.csv', index=False)
energy_data.head(15)

,time,location_specific,energy_MWh,week
0,2021-12-30 00:00:00+00:00,200 Olsson,19.716,1
1,2022-01-06 00:00:00+00:00,200 Olsson,27.210,2
2,2022-01-13 00:00:00+00:00,200 Olsson,27.585,3
3,2022-01-20 00:00:00+00:00,200 Olsson,27.463,4
4,2022-01-27 00:00:00+00:00,200 Olsson,29.105,5
5,2022-02-03 00:00:00+00:00,200 Olsson,29.672,6
6,2022-02-10 00:00:00+00:00,200 Olsson,29.882,7
7,2022-02-17 00:00:00+00:00,200 Olsson,30.105,8
8,2022-02-24 00:00:00+00:00,200 Olsson,30.715,9
9,2022-03-03 00:00:00+00:00,200 Olsson,31.237,10


In [34]:
energy_data.shape

(8036, 4)

In [35]:
# testing number of weeks based on 41 sensors...should be 195 or 196 weeks
8036 / 41

196.0